In [4]:
%matplotlib qt
import matplotlib.pyplot as plt
import hyperspy.api as hs
import pyxem as pxm
import numpy as np
from pathlib import Path
from diffsims.utils.sim_utils import get_electron_wavelength

In [2]:
import numpy as np
a = 4.04
hkl = np.array([8, 0, 0])
g = np.sqrt(np.sum(hkl**2/a**2))

In [3]:
print(g)

1.9801980198019802


In [5]:
datapath = Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\2021_10_06_2xxx_24h_250C\Preprocessed_data\SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3_preprocessed.hspy')
signal = hs.load(datapath, lazy=True)

In [7]:
signal.metadata.Preprocessing.Masks

└── Diffraction
    ├── cutoff = <LazySignal2D, title: <0.7001057239470767 $A^{-1}$ mask, dimensions: (|128, 128)>
    ├── direct_beam = <LazySignal2D, title: >0.15949382751461458 $A^{-1}$ mask, dimensions: (|128, 128)>
    └── reflections = <LazySignal2D, title: Reflection mask, dimensions: (|128, 128)>

In [12]:
mask = masks[0]

In [17]:
mask[1]

<LazySignal2D, title: <0.7001057239470767 $A^{-1}$ mask, dimensions: (|128, 128)>

In [18]:
masks = [mask[1] for mask in signal.metadata.Preprocessing.Masks.Diffraction]
print(masks)

[<LazySignal2D, title: <0.7001057239470767 $A^{-1}$ mask, dimensions: (|128, 128)>, <LazySignal2D, title: >0.15949382751461458 $A^{-1}$ mask, dimensions: (|128, 128)>, <LazySignal2D, title: Reflection mask, dimensions: (|128, 128)>]


In [30]:
signal.metadata.Preprocessing.Masks.diffraction

AttributeError: diffraction

In [31]:
fig, ax = plt.subplots()
ax.imshow(diffmask)

In [24]:
diffmask

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [2]:
datapath = Path(r'C:\Users\emilc\OneDrive - NTNU\NORTEM\Data\2021_10_06_2xxx_24h_250C\Site2\SPED_600x600x12_10x10_4p63x4p63_1deg_100Hz_CL12cm_NBD_alpha5_spot1p3.hspy')
raw_signal = hs.load(datapath, lazy=True)
preprocessed_signal = hs.load(datapath.with_name(f'{datapath.stem}_preprocessed.hspy'), lazy=True)

C:\Users\emilc\Miniconda3\envs\pyxem0.14.1\lib\site-packages\hyperspy\misc\utils.py:471: VisibleDeprecationWarning: Use of the `binned` attribute in metadata is going to be deprecated in v2.0. Set the `axis.is_binned` attribute instead. 
  warnings.warn(
C:\Users\emilc\Miniconda3\envs\pyxem0.14.1\lib\site-packages\hyperspy\io.py:560: VisibleDeprecationWarning: Loading old file version. The binned attribute has been moved from metadata.Signal to axis.is_binned. Setting this attribute for all signal axes instead.
  warnings.warn('Loading old file version. The binned attribute '


In [3]:
shifts = preprocessed_signal.metadata.Preprocessing.Centering.Shifts
raw_signal = raw_signal.inav[0:512, 0:512]
raw_signal.shift_diffraction(shifts.isig[0], shifts.isig[1], inplace=True)

In [14]:
mean = raw_signal.mean(axis=[0, 1])
mean.compute()

[########################################] | 100% Completed |  9min 30.1s


In [15]:
mean.plot(norm='symlog')
lines = {i: hs.roi.Line2DROI(0, 0, 0.1) for i in range(2)}
[lines[line].add_widget(mean) for line in lines]

In [19]:
sep='\n'
print(sep.join([f'Line {line}: ({lines[line].x1}, {lines[line].y1}) <-> ({lines[line].x2}, {lines[line].y2}' for line in lines]))

Line 0: (-0.9459047447218045, 0.5446118227186147) <-> (0.9554593381028329, -0.5541664160996431
Line 1: (-0.5255026359565581, -0.9363501513407761) <-> (0.5637210094806715, 0.984123118245918


In [21]:
profiles = {line: lines[line](mean) for line in lines}
[profiles[profile].plot(norm='log') for profile in profiles]
points = {profile: [hs.roi.Point1DROI(0), hs.roi.Point1DROI(0)] for profile in profiles}
[[point.add_widget(profiles[profile]) for point in points[profile]] for profile in profiles]

[[<hyperspy.drawing._widgets.vertical_line.VerticalLineWidget at 0x1e385a16140>,

In [23]:
sep='\n'
print(sep.join([f'Profile {profile} peaks: {[points[profile][i].value for i in range(len(points[profile]))]}' for profile in profiles]))

Profile 0 peaks: [0.1045723284022796, 2.081939992736294]
Profile 1 peaks: [0.08528197052928965, 2.056243067206206]


In [28]:
len(points[0])

2

In [33]:
lengths = {profile: (np.max([points[profile][i].value for i in range(len(points[profile]))]) - np.min([points[profile][i].value for i in range(len(points[profile]))])) / profiles[profile].axes_manager[-1].scale for profile in profiles}

sep = '\n'
print(sep.join([f'Profile {profile} peak to peak distance: {lengths[profile]}' for profile in profiles]))

hkl_values = {profile: np.array([8, 0, 0]) for profile in profiles}
a = 4.04
g_values = {profile: np.sqrt(np.sum((hkl_values[profile]/a)**2)) for profile in profiles}
scales = {profile: g_values[profile] / lengths[profile] for profile in profiles}
sep='\n'
print(sep.join([f'Profile {profile} scale: {scales[profile]}' for profile in profiles]))

print(f'Mean calibration: {np.mean(list(scales.values()))}')
print(f'std calibration: {np.std(list(scales.values()))}')

scale = np.round(np.mean(list(scales.values())), decimals=5)
print(f'Using mean calibration as calibration: {scale:.05f}')

binning_factors = [raw_signal.axes_manager.signal_shape[i]/preprocessed_signal.axes_manager.signal_shape[i] for i in range(2)]
print(f'Preprocessed signal is binned with scales {binning_factors}')
rescaled_scales = [scale*binning_factor for binning_factor in binning_factors]
print(f'Preprocessed signal has calibration {rescaled_scales}')
for i, scale in enumerate(rescaled_scales):
    preprocessed_signal.axes_manager[preprocessed_signal.axes_manager.navigation_dimension+i].scale = scale

Profile 0 peak to peak distance: 207.99999999999997
Profile 1 peak to peak distance: 208.00000000000003
Profile 0 scale: 0.009520182787509521
Profile 1 scale: 0.009520182787509518
Mean calibration: 0.00952018278750952
std calibration: 1.734723475976807e-18
Using mean calibration as calibration: 0.00952
Preprocessed signal is binned with scales [2.0, 2.0]
Preprocessed signal has calibration [0.01904, 0.01904]


In [34]:
preprocessed_signal.axes_manager

Navigation axis name,size,index,offset,scale,units
x,512,0,0.0,4.63,nm
y,512,0,0.0,4.63,nm
Signal axis name,size,,offset,scale,units
kx,128,,-1.2138,0.01904,$A^{-1}$
ky,128,,-1.2138,0.01904,$A^{-1}$


In [4]:
from skimage.feature import blob_log

## Mask reflections

In [5]:
binned = raw_signal.rebin(scale=(1, 1, 2, 2))

In [ ]:
data = signal.max(axis=[0, 1, 2, 3]).data
data.compute()
print(data)

In [ ]:
print(data)

In [ ]:
print('test')

In [6]:
#image = (signal*(2**16-1)).mean(axis=[0, 1])
image = binned.mean(axis=[0, 1])
image.compute()

[########################################] | 100% Completed |  8min 51.2s


In [7]:
im = image.deepcopy()

In [40]:
image.change_dtype('uint16')

In [55]:
2**16

65536

In [52]:
image.max(axis=[0, 1]).data

array([65106.0457243])

In [10]:
a = 4.04
minimum_r = 8 #
cutoff_hkl = np.array([2, 2, 0]) #Make a mask with cutoff at a given HKL
cuton_mrad = 4 #Make a mask that cutsoff everything up a certain mrad

#Set up mask arrays
nx, ny = image.axes_manager.signal_shape
mask = np.zeros((nx, ny), dtype=bool)
direct_beam_mask = np.zeros((nx, ny), dtype=bool)
cutoff_mask = np.zeros((nx, ny), dtype=bool)

#Setting cutoffs
cutoff_g = np.sqrt(np.sum(cutoff_hkl**2 / a**2))
cuton_k = cuton_mrad / 1000 / get_electron_wavelength(image.metadata.Acquisition_instrument.TEM.beam_energy/1000)
print(f'Minimum scattering vector: {cuton_k} {image.axes_manager[0].units}\nMaximum scattering vector: {cutoff_g} {image.axes_manager[0].units}')
X, Y = np.meshgrid(image.axes_manager[0].axis, image.axes_manager[1].axis)
#Set outer cutoff
R = np.sqrt(X**2 + Y**2)
cutoff_mask[R>=cutoff_g] = True
#Set inner cutoff
R = np.sqrt(X**2 + Y**2)
direct_beam_mask[R<=cuton_k] = True

#Mask reflections
blob_kwargs = {
    'min_sigma': 1,
    'max_sigma': 15,
    'num_sigma': 100,
    'overlap': 0,
    'threshold': 1.5E1,
}

# Look for blobs (reflections)
blobs = blob_log(image.data, **blob_kwargs)
print(len(blobs))
xs, ys = np.arange(0, nx), np.arange(0, ny)
X, Y = np.meshgrid(xs, ys)
for blob in blobs:
    y, x, r = blob  # x and y axes are flipped in hyperspy compared to numpy
    r = np.sqrt(2) * r  # Scale blob radius to appear more like a real radius
    r = max([minimum_r, r])  # Make sure that the radius is at least the specified minimum radius
    R = np.sqrt((X - x) ** 2 + (Y - y) ** 2)
    mask[R < r] = True

fig, axes = plt.subplots(nrows=1, ncols=6, sharex=True, sharey=True)
axes[0].imshow(image.data)
axes[1].imshow(mask)
axes[2].imshow(direct_beam_mask)
axes[3].imshow(cutoff_mask)
axes[4].imshow((image.data * ~(mask + direct_beam_mask))**0.256)
axes[5].imshow((image.data * ~cutoff_mask)**0.256)

Minimum scattering vector: 0.15949382751461458 $A^{-1}$
Maximum scattering vector: 0.7001057239470767 $A^{-1}$
25
